# An Overview of a regression assignment 
Attempting to fit Life Expectancy.


### 0. Imports

: 

In [ ]:
#data processing
import pandas as pd
import numpy as np

#data visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#Machine learning library
import sklearn

import warnings
warnings.filterwarnings("ignore")


### 1. Introducing the Life Expectancy dataset -- Exploratory Data Analysis (EDA)
##### The goal of this phase is to understand the different features and relations between them and w.r.t. the target feature

Loading the CSV dataset

In [ ]:
dtf = pd.read_csv("./data/Life Expectancy Data.csv",index_col='Country')

In [ ]:
dtf.columns = dtf.columns.str.strip()
dtf = dtf.drop(columns=['Hepatitis B', 'GDP', 'Diphtheria','percentage expenditure'])


In [ ]:
dtf.head()

In [ ]:
dtf.rename(columns = lambda c: c.replace(' ', ''), inplace=True)
dtf.describe()


#### Examining the target feature - "Lifexpectancy": Using a histogram
Our main goal at this point is to examine the highest correlations (in absolute value - also taking into account strong negative correlations) between a feature and the target col (Life Expectancy).

In [ ]:
min_val_corr = 0.3

df_numeric = dtf.drop(columns=dtf.select_dtypes(include=['object']).columns)
target = 'Lifeexpectancy'
corr = df_numeric.corr()
corr_abs = corr.abs()
ser_corr = corr_abs.nlargest(len(df_numeric),target)[target]
cols_above_corr_limit = list(ser_corr[ser_corr.values > min_val_corr].index)
df_corr = df_numeric[cols_above_corr_limit]

correlations = df_corr.corr()[target].drop(target)

# Create a bar plot of the correlations with the target
plt.figure(figsize=(10, 6))
sns.barplot(x=correlations.index, y=correlations.values, palette='inferno')
plt.title('Correlation with Target Column')
plt.xlabel('Features')
plt.ylabel('Correlation')
plt.xticks(rotation=45)
plt.show()

The histogram above wel demonstrate that the features 'schooling' and'Income'  have high positive correlation with life expectancy. 
On the other hand 'AdultMortality' has strong negative correlation to the target feature.

In [ ]:
df_numeric.hist(figsize=(20, 16), bins=50, xlabelsize=8, ylabelsize=8)
plt.show()

The visualization above demonstrate a number of key points:
1. The distribution of thinnes 1-19 years is almost identical to theoe of thinnes 5-9 years. Thus we would drop the latter.
2. It seems like almost 50% of the IBM values in the dtaset are 50 and higher. Which is extremly high (According to BMI score calcuator), thus we'd probably drop that column, since its distribution in the dataset isn't realistic.
3. Life Expectancy's distribution is almost normal around 73-74.


### Now we'd like to take a deeper loook into the Lifeexpectancy column on our dataset with respect to the categorical 'Status' feature (Developed/Developing).

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
status_counts = dtf['Status'].value_counts()
axes[0].pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', startangle=90, colors=['#0fffc7','#6693ff','#99ff99'])

sns.boxplot(x='Status', y='Lifeexpectancy', data=dtf.dropna(), ax = axes[1])
axes[1].set_title('Boxplot of Lifeexpectancy by Status')
dtf =  dtf.dropna()

plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
sns.regplot(data=dtf,x='Year',y='Lifeexpectancy')
# Show the plot
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(19, 5))
dtf.groupby(pd.cut(dtf['Schooling'],5))['Lifeexpectancy'].mean().plot(kind='line',ax=axes[0])
dtf.groupby(pd.cut(dtf['Incomecompositionofresources'],11))['Lifeexpectancy'].mean().plot(kind='line',ax=axes[1])
# Show the plot
plt.show()

#### Examining Correlations to the target feature:

#### Visualzing relevant feature-pair relations:

While the plot above is difficult to read, we can bin the year column then group by and view the mean price

### 2 - Cleaning and Preprocessing

A. Split to Train and Test, then see the target feature distributions:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
dtf_train, dtf_test = train_test_split(dtf, 
                      test_size=0.25)

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
dtf_train.Lifeexpectancy.hist(ax=ax)
dtf_test.Lifeexpectancy.hist(ax=ax)
plt.show()

Concat the one-hot attributes and drop the original

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# convert categorical feature into one-hot encoding representation
dummy_train = pd.get_dummies(dtf_train['Status'], prefix='Status')
dummy_test = pd.get_dummies(dtf_test['Status'], prefix='Status')
# update test and train datasets with the new column of 'Status' Feature
dtf_train = pd.concat([dtf_train, dummy_train], axis=1).drop(columns='Status')
dtf_test = pd.concat([dtf_test, dummy_test], axis=1).drop(columns='Status')


### 3. Create a Baseline Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
#separate X from y
X_train = dtf_train.drop('Lifeexpectancy',axis=1)
X_test = dtf_test.drop('Lifeexpectancy',axis=1)

y_train = dtf_train['Lifeexpectancy']
y_test = dtf_test['Lifeexpectancy']




#### Train a linear regression model

In [ ]:
import xgboost as xgb
model = xgb.XGBRegressor()

In [ ]:
prediction = model.fit(X_train,y_train).predict(X_test)

### 4. Evaluate how good is the model

Many metrics exist for evaluating the regression over the test data. 


In [ ]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_percentage_error,mean_absolute_error

Let's start with r^2: R Squared is the squared sum of differences from the actual values and the predicted values, divided by the squared differences from the mean (i.e var*n)

In [ ]:
r2_score(y_test, prediction)

<br> What is a good R2 score and how do we improve it? This is not an ML class, a better question is what does the score "means"
<br>
High R2 means that the model "explains" a lot of the variance, i.e. that the behaviour is "predicted".
It *doesn't* directly imply whether the model is right!
<br>
Let's see more metrics:

In [ ]:
print("Mean Absolute Perc Error (Σ(|y - pred|/y)/n):","{:,.3f}".format(mean_absolute_percentage_error(y_test,prediction)))
print("Mean Absolute Error (Σ|y - pred|/n):", "{:,.0f}".format(mean_absolute_error(y_test, prediction)))
print("Root Mean Squared Error (sqrt(Σ(y - pred)^2/n)):", "{:,.0f}".format(np.sqrt(mean_squared_error(y_test, prediction))))

## residuals
residuals = y_test - prediction
max_error = residuals.abs().max()
max_idx = residuals[residuals==max_error]
#max_true, max_pred = y_test.loc[max_idx], prediction[max_idx]
print("Max Error:", "{:,.0f}".format(max_error))

We now plot the true values against the predicted values. 
<br> In the regression line, the predicted values are always on the function y=x

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.scatterplot(x= prediction,y = y_test,ax=ax,color='blue')
sns.lineplot(x = prediction,y = prediction,ax=ax,color='black')
plt.title("model's test prediction")

We can already learn that our bigger mistakes are when the sale price is larger.
<br>
Lets take a deeper looks into the 'residuals'

In [ ]:
residuals.hist()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(16,5))
sns.scatterplot(x=prediction,y=residuals,ax=ax[0])
sns.lineplot(prediction,0,ax=ax[0],color='black')
ax[0].set_title("Residuals (Abs)")
sns.scatterplot(prediction,residuals/y_test,ax=ax[1])
sns.lineplot(x=prediction,y=0,ax=ax[1],color='black')
ax[1].set_title("Residuals (%)")

So we can see that we have "small" mistakes and "big" mistakes. Let's look into that:

In [ ]:
rel_res=residuals/y_test
rel_res=rel_res.abs()

How many time did our model achieve low estimation error (error less than 5%)?:

In [ ]:
len(rel_res[rel_res<0.05])/len(rel_res)

How about more than 20%?

In [ ]:
len(rel_res[rel_res>0.2])/len(rel_res)

### 4. Model Explainability

We are first interested in feature importance.
<br>
In the simple linear regression model, we can look at the learned coefficients:

In [ ]:
print("Model coefficients:\n")
for i in range(len(X_train.columns)):
    print(X_train.columns[i], "=", model.coef_[i].round(4))

However, since our features are not normalized, it is hard to assess which ones are the most important.
<br>
For that, we use SHAP

In [ ]:
import shap

In [ ]:
shap_sample = X_train.sample(500)

In [ ]:
explainer = shap.Explainer(model.predict, shap_sample)
shap_values = explainer(shap_sample)

In [ ]:
dtf.columns

In [ ]:
shap.plots.beeswarm(shap_values)

Remember that we can also do that to explain the prediction of a single element:

In [ ]:
test_shap_values = explainer(X_test)

What was our biggest relative error?

In [ ]:
rel_res[rel_res==rel_res.max()]


Our model predicted:

In [ ]:
max_id = rel_res[rel_res==rel_res.max()].index[0]
pred_series=pd.Series(prediction,index=rel_res.index)
pred_series[max_id]

And the real price was:

In [ ]:
y_test[max_id]

Let's look at this problematic point:

In [ ]:
X_test.loc[max_id]


In [ ]:
max_ordinal_id= X_test.index.get_loc(max_id)

In [ ]:
#sns.boxenplot(dtf.GrLivArea)

In [ ]:
shap.plots.waterfall(test_shap_values[max_ordinal_id])

In [ ]:
len(residuals[residuals>50000])

In [ ]:
bad_examples = X_test.loc[residuals[residuals>50000].index]
bad_examples_shap_values = explainer(bad_examples)
shap.plots.beeswarm(bad_examples_shap_values)

To get even a deeper understanding regarding our model's mistakes, we can compare the distributions of our mistakes to good predictions

In [ ]:
over_estimates = X_test.loc[residuals[residuals>50000].index]

In [ ]:
good_estimates = X_test.loc[rel_res[rel_res<0.05].index]

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
good_estimates.GrLivArea.hist(ax=ax,color='blue')
over_estimates.GrLivArea.hist(ax=ax,color='red')

We can see that our overestimas contains "larger" apartments.
<br> While the model correctly understood that larger apartments are often more expensive, this is not always correct! 

In [ ]:
len(good_estimates)

In [ ]:
len(over_estimates)